In [1]:
from pathlib import Path

import fsspec
import s3fs
import xarray as xr
import zarr

In [3]:
s3cfg = dict([l.split(' = ') for l in Path('/home/markmuetz/.s3cfg').read_text().split('\n') if l])

In [7]:
jasmin_s3 = s3fs.S3FileSystem(
    anon=False, secret=s3cfg['secret_key'],
    key=s3cfg['access_key'],
    client_kwargs={'endpoint_url': 'https://hackathon-o.s3-ext.jc.rl.ac.uk'}
)
jasmin_s3.ls('data')

['data/20200101T0000Z_ph2496.pp', 'data/hpz0', 'data/zarr_example.zarr']

In [8]:
store= s3fs.S3Map(root='s3://data/hpz0/glm.n2560_RAL3p3.apverd_20200120T00.air_temperature.hpz0.zarr', s3=jasmin_s3, check=False)


In [9]:
dsT2 = xr.open_zarr(store=store)

In [12]:
dsT2.air_temperature.sum().compute()

<xarray.DataArray 'air_temperature' ()> Size: 8B
array(376112.32569437)
Coordinates:
    forecast_reference_time  datetime64[ns] 8B 2020-01-20

In [13]:
# Have to use access key/secret.
jasmin_s3 = s3fs.S3FileSystem(
    anon=True,
    client_kwargs={'endpoint_url': 'https://hackathon-o.s3-ext.jc.rl.ac.uk'}
)
jasmin_s3.ls('data')

PermissionError: Access Denied